In [14]:
# !pip install openpyxl

In [105]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [106]:
df = pd.read_csv("/data/Augmented_data.csv")
df.shape

(234501, 18)

In [107]:
df[df["Exited"] == 1].shape[0] /df.shape[0]

0.14290770615050682

In [110]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15572489,HEALEY,1720.007287,Spain,Female,73,29,393987.727351,1,1,1,1.0,0,0,3,DIAMOND,2531.0,1-6-2022
1,15800464,PARTIDA,1518.782461,Spain,Male,53,15,274502.817582,2,1,0,1.0,1,0,4,SILVER,1725.0,1-2-2022
2,15580134,Crawford,502.950000,Spain,Male,27,2,184535.891500,1,1,1,1.0,0,0,3,DIAMOND,711.0,1-1-2020
3,15074588,PINKERTON,1468.954354,Spain,Male,49,15,0.000000,1,1,0,1.0,0,0,5,SILVER,1512.0,1-1-2022
4,15982065,ABERNATHY,1683.914742,France,Male,49,30,0.000000,2,1,0,1.0,1,0,3,DIAMOND,1296.0,1-6-2022


In [109]:
cus_st = {}
for cus, st in zip(df["CustomerId"],df["Geography"]):
    if cus not in cus_st:
        cus_st[cus] = [st]
    else:
        if st not in cus_st:
            cus_st[cus].append(st)

In [95]:
df["Geography"] =  df[["Geography","CustomerId"]].progress_apply(lambda x: cus_st[x[1]][0], axis =1)

100%|██████████| 234501/234501 [00:01<00:00, 196988.81it/s]


In [99]:
cities = pd.read_excel("US_cities.xlsx")
# companies = pd.read_csv("us_companies.csv")

In [100]:
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)

In [101]:
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())

100%|██████████| 234501/234501 [00:00<00:00, 999394.90it/s] 


In [102]:
state_mapper = {}
def get_state(x):
    st = x[0]
    cust_id = x[1]
    if cust_id not in state_mapper:
        state = np.random.choice(list(state_city.keys()))
        state_mapper[cust_id] = {st: state}
        return state
    else:
        return state_mapper[cust_id][st]
    
df["Geography"] = df[["Geography","CustomerId"]].progress_apply(get_state, axis=1)

100%|██████████| 234501/234501 [00:02<00:00, 82030.27it/s] 


In [104]:
df["Geography"].unique()

array(['District of Columbia', 'Iowa', 'Massachusetts', 'Wyoming',
       'Georgia', 'New Mexico', 'South Dakota', 'Washington', 'Louisiana',
       'Rhode Island', 'Idaho', 'Pennsylvania', 'North Carolina', 'Ohio',
       'Connecticut', 'Minnesota', 'Hawaii', 'Vermont', 'Oregon',
       'Oklahoma', 'Alabama', 'West Virginia', 'Arizona', 'Mississippi',
       'Tennessee', 'Montana', 'Wisconsin', 'Kansas', 'Arkansas',
       'Colorado', 'Kentucky', 'Nevada', 'New Jersey', 'California',
       'Alaska', 'Missouri', 'Maryland', 'Florida', 'Texas',
       'South Carolina', 'Indiana', 'Michigan', 'Maine', 'North Dakota',
       'New York', 'Utah', 'Virginia', 'New Hampshire', 'Nebraska',
       'Illinois', 'Delaware'], dtype=object)

In [73]:
cus_city_mapper = {}
def get_state(x):
    st = x[0]
    cust_id = x[1]
    if cust_id not in cus_city_mapper:
        ct = np.random.choice(state_city[st])
        cus_city_mapper[cust_id] = {st: ct}
        return ct
    else:
        return cus_city_mapper[cust_id][st]
    
    
df["City"] = df[["Geography","CustomerId"]].progress_apply(lambda x: get_state(x), axis =1 )

  1%|          | 2051/234501 [00:00<00:07, 32843.44it/s]


KeyError: 'Michigan'

In [57]:
df["month"] = df["date"].progress_apply(lambda x: x.split("-")[1])
df["year"] = df["date"].progress_apply(lambda x: x.split("-")[-1])

100%|██████████| 234501/234501 [00:00<00:00, 1090821.31it/s]


In [59]:
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: round(x,0))

100%|██████████| 234501/234501 [00:00<00:00, 719684.61it/s]


In [62]:
df["Account_type"] = df["Tenure"].progress_apply(lambda x: "Gold" if x > 15 else "Premium" if x <= 15 and x >=8 else "Normal")

100%|██████████| 234501/234501 [00:00<00:00, 1253114.71it/s]


In [63]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date,City,month,year,Account_type
0,15572489,Healey,1720.0,New York,Female,73,29,393987.727351,1,1,...,0,0,3,DIAMOND,2531.0,1-6-2022,Buffalo,6,2022,Gold
1,15800464,Partida,1519.0,New York,Male,53,15,274502.817582,2,1,...,1,0,4,SILVER,1725.0,1-2-2022,Syracuse,2,2022,Premium
2,15580134,Crawford,503.0,New York,Male,27,2,184535.891500,1,1,...,0,0,3,DIAMOND,711.0,1-1-2020,Syracuse,1,2020,Normal
3,15074588,Pinkerton,1469.0,New York,Male,49,15,0.000000,1,1,...,0,0,5,SILVER,1512.0,1-1-2022,Binghamton,1,2022,Premium
4,15982065,Abernathy,1684.0,Nebraska,Male,49,30,0.000000,2,1,...,1,0,3,DIAMOND,1296.0,1-6-2022,Lincoln,6,2022,Gold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234496,15244801,King,1638.0,Nebraska,Male,65,37,0.000000,1,1,...,0,0,5,GOLD,643.0,1-10-2022,Bellevue,10,2022,Gold
234497,15047568,Medley,612.0,Nebraska,Male,32,1,172743.096102,1,0,...,0,0,1,DIAMOND,722.0,1-4-2020,Omaha,4,2020,Normal
234498,15617037,Wyman,856.0,New York,Male,37,2,213987.894170,1,1,...,0,0,5,SILVER,870.0,1-10-2020,Yonkers,10,2020,Normal
234499,15788634,Romani,827.0,New York,Female,37,2,124219.939284,1,0,...,0,0,5,DIAMOND,1018.0,1-2-2020,Mount Vernon,2,2020,Normal
